# PyTorch Modeling for [LEGO Minifigures Classification](https://www.kaggle.com/ihelon/lego-minifigures-classification) dataset

This is the guide about using pretrained models in **PyTorch** and logging experiments with [**Neptune.ai**](https://neptune.ai/).   
We will use MobileNetV2 model (with options to change the base model) to predict which Lego Figure is in the image.


![](https://i.imgur.com/4cPQlEN.jpg)

<a id="top"></a>

<div class="list-group" id="list-tab" role="tablist">
<h3 class="list-group-item list-group-item-action active" data-toggle="list" style='background:blue; border:0' role="tab" aria-controls="home"><center>Quick Navigation</center></h3>

* [1. Configurations](#1)
* [2. Data reading](#2)
* [3. Data sample retriever class](#3)
* [4. Augmentations](#4)
* [5. Train and valid retrievers](#5)   
* [6. Batch train and valid data loaders](#6)
* [7. Data visualizations](#7)
* [8. Model creating class](#8)
* [9. Loss and metric calculating classes](#9)
* [10. Model training class](#10)
* [11. Model training](#11)
* [12. Final validation check](#12)
* [13. Error analysis - Confusion matrix](#13)
* [14. Error analysis - Misclassified samples](#14)


<a id="1"></a>
<h2 style='background:blue; border:0; color:white'><center>Configurations<center><h2>

We need to turn internet on to install client for Neptune.ai

In [ ]:
!pip install -q -U pip
!pip install -q neptune-client

In [ ]:
import os
import math
import time
import random

import yaml
import numpy as np
import pandas as pd
import cv2
import matplotlib.pyplot as plt
import seaborn as sn
import albumentations as A
import torch
from torch.utils import data as torch_data
from torch import nn as torch_nn
from torch.nn import functional as torch_functional
import torchvision
from sklearn import metrics as sk_metrics
from sklearn import model_selection as sk_model_selection
import neptune

For using Neptune.ai you need to specify special token (you can get it from the Neptune client).   
For security reasons we need to create private dataset and set there our secret token and project name as in the client on Neptune.   
For example we can create neptune-config dataset and specify config.yaml there

In [ ]:
def load_neptune_config(config_path):
    """
    Load info about neptune from .yaml file with the next format:
    
    api_token: <your secret token from neptune>
    project_qualified_name: <your neptune project name>
    
    """
    with open(config_path) as file:
        neptune_config = yaml.load(file, Loader=yaml.FullLoader)
    return neptune_config


neptune_config = load_neptune_config("../input/neptune-config/config.yaml")

In [ ]:
config = {
    "seed": 42,
    
    "valid_size": 0.3,
    "image_size": (512, 512),
    
    "train_batch_size": 4,
    "valid_batch_size": 1,
    "test_batch_size": 1,
    
    "model": "mobilenet_v2",
    
    "max_epochs": 50,
    "model_save_path": "model-best.torch",
    "patience_stop": 3,
    
    "optimizer": "adam",
    "adam_lr": 0.0001,
    
    "criterion": "cross_entropy",
}

neptune_settings = {
    "active": True,
    "log_images": True,
    "log_artifacts": False, # set True to save model weights
}

if neptune_settings["active"]:
    
    neptune.init(
        api_token=neptune_config["api_token"], 
        project_qualified_name=neptune_config["project_qualified_name"],
    )

    neptune.create_experiment(
        name=f"{config['model']}_experiment",
        params=config,
        upload_source_files=os.listdir(os.getcwd()),
    )

    neptune.append_tag("data-version: 20")

In [ ]:
# The directory to the dataset
BASE_DIR = '../input/lego-minifigures-classification/'
PATH_INDEX = os.path.join(BASE_DIR, "index.csv")
PATH_TEST = os.path.join(BASE_DIR, "test.csv")
PATH_METADATA = os.path.join(BASE_DIR, "metadata.csv")

In [ ]:
# Try to set random seet that our experiment repeated between (We have some problem to set seed with GPU)
def set_seed(seed):
    random.seed(seed)
    os.environ["PYTHONHASHSEED"] = str(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)
    if torch.cuda.is_available():
        torch.cuda.manual_seed_all(seed)
        torch.backends.cudnn.deterministic = True


set_seed(config["seed"])

<a id="2"></a>
<h2 style='background:blue; border:0; color:white'><center>Data reading<center><h2>

In [ ]:
# Read information about dataset
df = pd.read_csv(PATH_INDEX)

tmp_train, tmp_valid = sk_model_selection.train_test_split(
    df, 
    test_size=config["valid_size"], 
    random_state=config["seed"], 
    stratify=df['class_id'],
)


def get_paths_and_targets(tmp_df):
    # Get file paths
    paths = tmp_df["path"].values
    # Create full paths (base dir + concrete file name)
    paths = list(
        map(
            lambda x: os.path.join(BASE_DIR, x), 
            paths
        )
    )
    # Get labels
    targets = tmp_df["class_id"].values
    
    return paths, targets


# Get train file paths and targets
train_paths, train_targets = get_paths_and_targets(tmp_train)

# Get valid file paths and targets
valid_paths, valid_targets = get_paths_and_targets(tmp_valid)

df_test = pd.read_csv(PATH_TEST)
# Get test file paths and targets
test_paths, test_targets = get_paths_and_targets(df_test)

In [ ]:
# Calculate the total number of classes in the dataset (len of unique labels in data)
df_metadata = pd.read_csv(PATH_METADATA)
n_classes = df_metadata.shape[0]
print("Number of classes: ", n_classes)

<a id="3"></a>
<h2 style='background:blue; border:0; color:white'><center>Data sample retriever class<center><h2>

In [ ]:
class DataRetriever(torch_data.Dataset):
    def __init__(
        self, 
        paths, 
        targets, 
        image_size,
        transforms=None,
        preprocess=None,
    ):
        self.paths = paths
        self.targets = targets
        self.image_size = image_size
        self.transforms = transforms
        self.preprocess = preprocess
          
    def __len__(self):
        return len(self.targets)
    
    def __getitem__(self, index):
        img = cv2.imread(self.paths[index])
        img = cv2.resize(img, self.image_size)
        img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
        if self.transforms:
            img = self.transforms(image=img)['image']
        if self.preprocess:
            img = self.preprocess(img)
        
        y = torch.tensor(self.targets[index] - 1, dtype=torch.long)
            
        return {'X': img, 'y': y}

<a id="4"></a>
<h2 style='background:blue; border:0; color:white'><center>Augmentations<center><h2>

In [ ]:
def get_train_transforms():
    return A.Compose(
        [
            A.Rotate(limit=30, border_mode=cv2.BORDER_REPLICATE, p=0.5),
            A.Cutout(num_holes=8, max_h_size=20, max_w_size=20, fill_value=0, p=0.25),
            A.Cutout(num_holes=8, max_h_size=20, max_w_size=20, fill_value=1, p=0.25),
            A.HorizontalFlip(p=0.5),
            A.RandomContrast(limit=(-0.3, 0.3), p=0.5),
            A.RandomBrightness(limit=(-0.4, 0.4), p=0.5),
            A.Blur(p=0.25),
        ], 
        p=1.0
    )

def get_preprocess():
    return torchvision.transforms.Compose(
        [
            torchvision.transforms.ToTensor(),
            torchvision.transforms.Normalize(
                mean=[0.485, 0.456, 0.406], 
                std=[0.229, 0.224, 0.225]
            ),
        ]
    )

<a id="5"></a>
<h2 style='background:blue; border:0; color:white'><center>Train and valid retrievers<center><h2>

In [ ]:
train_data_retriever = DataRetriever(
    train_paths, 
    train_targets, 
    image_size=config["image_size"],
    transforms=get_train_transforms(),
    preprocess=get_preprocess(),
)

valid_data_retriever = DataRetriever(
    valid_paths, 
    valid_targets, 
    image_size=config["image_size"],
    preprocess=get_preprocess(),
)

test_data_retriever = DataRetriever(
    test_paths, 
    test_targets, 
    image_size=config["image_size"],
    preprocess=get_preprocess(),
)

<a id="6"></a>
<h2 style='background:blue; border:0; color:white'><center>Batch train and valid data loaders<center><h2>

In [ ]:
train_loader = torch_data.DataLoader(
    train_data_retriever,
    batch_size=config["train_batch_size"],
    shuffle=True,
)

valid_loader = torch_data.DataLoader(
    valid_data_retriever, 
    batch_size=config["valid_batch_size"],
    shuffle=False,
)

test_loader = torch_data.DataLoader(
    test_data_retriever, 
    batch_size=config["test_batch_size"],
    shuffle=False,
)

<a id="7"></a>
<h2 style='background:blue; border:0; color:white'><center>Data visualizations<center><h2>

In [ ]:
def denormalize_image(image):
    return image * [0.229, 0.224, 0.225] + [0.485, 0.456, 0.406]

# Let's visualize some batches of the train data
fig = plt.figure(figsize=(16, 16))
for i_batch, batch in enumerate(train_loader):
    images, labels = batch["X"], batch["y"]
    for i in range(len(images)):
        plt.subplot(4, 4, 4 * i_batch + i + 1)
        plt.imshow(denormalize_image(images[i].permute(1, 2, 0).numpy()))
        plt.title(labels[i].numpy())
        plt.axis("off")
    if i_batch >= 3:
        break
        
if neptune_settings["active"] and neptune_settings["log_images"]:
    neptune.log_image("data-examples", fig)

In [ ]:
# Let's visualize some batches of the train data
fig = plt.figure(figsize=(16, 16))
for i_batch, batch in enumerate(valid_loader):
    images, labels = batch["X"], batch["y"]
    plt.subplot(4, 4, i_batch + 1)
    plt.imshow(denormalize_image(images[0].permute(1, 2, 0).numpy()))
    plt.title(labels[0].numpy())
    plt.axis("off")
    if i_batch >= 15:
        break
        
if neptune_settings["active"] and neptune_settings["log_images"]:
    neptune.log_image("data-examples", fig)

<a id="8"></a>
<h2 style='background:blue; border:0; color:white'><center>Model creating class<center><h2>

## We use simple MobileNetV2 model as a backbone

In [ ]:
def init_model_mobilenet_v2(n_classes):
    net = torch.hub.load("pytorch/vision:v0.6.0", "mobilenet_v2", pretrained=True)
    net.classifier = torch_nn.Linear(
        in_features=1280, 
        out_features=n_classes, 
        bias=True,
    )
    return net

def init_model_resnet18(n_classes):
    net = torch.hub.load('pytorch/vision:v0.6.0', 'resnet18', pretrained=True)
    net.classifier = torch_nn.Linear(
        in_features=512, 
        out_features=n_classes, 
        bias=True,
    )
    return net

def init_model_resnet101(n_classes):
    net = torch.hub.load('pytorch/vision:v0.6.0', 'resnet101', pretrained=True)
    net.classifier = torch_nn.Linear(
        in_features=2048, 
        out_features=n_classes, 
        bias=True,
    )
    return net

def init_model_vgg16(n_classes):
    net = torch.hub.load('pytorch/vision:v0.6.0', 'vgg16', pretrained=True)
    net.classifier[6] = torch_nn.Linear(
        in_features=4096, 
        out_features=n_classes, 
        bias=True,
    )
    return net

def init_model_resnext50_32x4d(n_classes):
    net = torch.hub.load('pytorch/vision:v0.6.0', 'resnext50_32x4d', pretrained=True)
    net.classifier = torch_nn.Linear(
        in_features=2048, 
        out_features=n_classes, 
        bias=True,
    )
    return net

<a id="9"></a>
<h2 style='background:blue; border:0; color:white'><center>Loss and metric calculating classes<center><h2>

In [ ]:
class LossMeter:
    def __init__(self):
        self.avg = 0
        self.n = 0

    def update(self, val):
        self.n += 1
        # incremental update
        self.avg = val / self.n + (self.n - 1) / self.n * self.avg

        
class AccMeter:
    def __init__(self):
        self.avg = 0
        self.n = 0
        
    def update(self, y_true, y_pred):
        y_true = y_true.cpu().numpy().astype(int)
        y_pred = y_pred.cpu().numpy().argmax(axis=1).astype(int)
        last_n = self.n
        self.n += len(y_true)
        true_count = np.sum(y_true == y_pred)
        # incremental update
        self.avg = true_count / self.n + last_n / self.n * self.avg

<a id="10"></a>
<h2 style='background:blue; border:0; color:white'><center>Model training class<center><h2>

In [ ]:
class Trainer:
    def __init__(
        self, 
        model, 
        device, 
        optimizer, 
        criterion, 
        loss_meter, 
        score_meter
    ):
        self.model = model
        self.device = device
        self.optimizer = optimizer
        self.criterion = criterion
        self.loss_meter = loss_meter
        self.score_meter = score_meter
        
        self.best_valid_score = -np.inf
        self.n_patience = 0
        
        self.messages = {
            "epoch": "[Epoch {}: {}] loss: {:.5f}, score: {:.5f}, time: {} s",
            "checkpoint": "The score improved from {:.5f} to {:.5f}. Save model to '{}'",
            "patience": "\nValid score didn't improve last {} epochs."
        }
    
    def fit(self, epochs, train_loader, valid_loader, save_path, patience):
        history = {
            "train_loss": [],
            "train_score": [],
            "valid_loss": [],
            "valid_score": [],
        }
        
        for n_epoch in range(1, epochs + 1):
            self.info_message("EPOCH: {}", n_epoch)
            
            train_loss, train_score, train_time = self.train_epoch(train_loader)
            valid_loss, valid_score, valid_time = self.valid_epoch(valid_loader)
            
            history["train_loss"].append(train_loss)
            history["train_score"].append(train_score)
            history["valid_loss"].append(valid_loss)
            history["valid_score"].append(valid_score)
            
            self.info_message(
                self.messages["epoch"], "Train", n_epoch, train_loss, train_score, train_time
            )
            if neptune_settings["active"]:
                neptune.log_metric(f'train_loss', train_loss)
                neptune.log_metric(f'train_accuracy', train_score)
            
            self.info_message(
                self.messages["epoch"], "Valid", n_epoch, valid_loss, valid_score, valid_time
            )
            if neptune_settings["active"]:
                neptune.log_metric(f'valid_loss', valid_loss)
                neptune.log_metric(f'valid_accuracy', valid_score)
            
            if self.best_valid_score < valid_score:
                self.info_message(
                    self.messages["checkpoint"], self.best_valid_score, valid_score, save_path
                )
                self.best_valid_score = valid_score
                self.save_model(n_epoch, save_path)
                self.n_patience = 0
            else:
                self.n_patience += 1
            
            if self.n_patience >= patience:
                self.info_message(self.messages["patience"], patience)
                break
        
        return history
            
    def train_epoch(self, train_loader):
        self.model.train()
        t = time.time()
        train_loss = self.loss_meter()
        train_score = self.score_meter()
        
        for step, batch in enumerate(train_loader, 1):
            images = batch["X"].to(self.device)
            targets = batch["y"].to(self.device)
            self.optimizer.zero_grad()
            outputs = self.model(images)

            loss = self.criterion(outputs, targets)
            loss.backward()

            train_loss.update(loss.detach().item())
            train_score.update(targets, outputs.detach())

            self.optimizer.step()
        
        return train_loss.avg, train_score.avg, int(time.time() - t)
    
    def valid_epoch(self, valid_loader):
        self.model.eval()
        t = time.time()
        valid_loss = self.loss_meter()
        valid_score = self.score_meter()

        for step, batch in enumerate(valid_loader, 1):
            with torch.no_grad():
                images = batch["X"].to(self.device)
                targets = batch["y"].to(self.device)

                outputs = self.model(images)
                loss = self.criterion(outputs, targets)

                valid_loss.update(loss.detach().item())
                valid_score.update(targets, outputs)
        
        return valid_loss.avg, valid_score.avg, int(time.time() - t)
    
    def save_model(self, n_epoch, save_path):
        torch.save(
            {
                "model_state_dict": self.model.state_dict(),
                "optimizer_state_dict": self.optimizer.state_dict(),
                "best_valid_score": self.best_valid_score,
                "n_epoch": n_epoch,
            },
            save_path,
        )
    
    @staticmethod
    def info_message(message, *args, end="\n"):
        print(message.format(*args), end=end)

<a id="11"></a>
<h2 style='background:blue; border:0; color:white'><center>Model training<center><h2>

In [ ]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

if config["model"] == "mobilenet_v2":
    model = init_model_mobilenet_v2(n_classes)
elif config["model"] == "resnet18":
    model = init_model_resnet18(n_classes)
elif config["model"] == "resnet101":
    model = init_model_resnet101(n_classes)
elif config["model"] == "vgg16":
    model = init_model_vgg16(n_classes)
elif config["model"] == "resnext50_32x4d":
    model = init_model_resnext50_32x4d(n_classes)
    
model.to(device)

if config["optimizer"] == "adam":
    optimizer = torch.optim.Adam(model.parameters(), lr=config["adam_lr"])

if config["criterion"] == "cross_entropy":
    criterion = torch_functional.cross_entropy

trainer = Trainer(
    model, 
    device, 
    optimizer, 
    criterion, 
    LossMeter, 
    AccMeter
)

history = trainer.fit(
    config["max_epochs"], 
    train_loader, 
    valid_loader, 
    config["model_save_path"], 
    config["patience_stop"],
)

if neptune_settings["active"] and neptune_settings["log_artifacts"]:
    neptune.log_artifact(config["model_save_path"])

In [ ]:
# Visualize train and valid loss 
plt.figure(figsize=(16, 6))
plt.subplot(1, 2, 1)
plt.plot(history['train_loss'], label='train loss')
plt.plot(history['valid_loss'], label='valid loss')
plt.xticks(fontsize=14)
plt.xlabel("Epoch number", fontsize=15)
plt.yticks(fontsize=14)
plt.ylabel("Loss value", fontsize=15)
plt.legend(fontsize=15)
plt.grid()

# Visualize train and valid accyracy 
plt.subplot(1, 2, 2)
plt.plot(history['train_score'], label='train acc')
plt.plot(history['valid_score'], label='valid acc')
plt.xticks(fontsize=14)
plt.xlabel("Epoch number", fontsize=15)
plt.yticks(fontsize=14)
plt.ylabel("Accuracy score", fontsize=15)
plt.legend(fontsize=15)
plt.grid();

<a id="12"></a>
<h2 style='background:blue; border:0; color:white'><center>Final validation check<center><h2>

In [ ]:
# Load the best model
checkpoint = torch.load(config["model_save_path"])

model.load_state_dict(checkpoint["model_state_dict"])
optimizer.load_state_dict(checkpoint["optimizer_state_dict"])
best_valid_score = checkpoint["best_valid_score"]
n_epoch = checkpoint["n_epoch"]

model.eval()

print(f"Best model valid score: {best_valid_score} ({n_epoch} epoch)")

In [ ]:
# Save the model predictions and true labels
y_pred = []
y_test = []
for batch in test_loader:
    y_pred.extend(model(batch['X'].to(device)).argmax(axis=-1).cpu().numpy())
    y_test.extend(batch['y'])

# Calculate needed metrics
test_accuracy = sk_metrics.accuracy_score(y_test, y_pred)
test_f1_macro = sk_metrics.f1_score(y_test, y_pred, average="macro")

print(f"Accuracy score on test data:\t{test_accuracy}")
print(f"Macro F1 score on test data:\t{test_f1_macro}")

if neptune_settings["active"]:
    neptune.log_metric(f'test_accuracy', test_accuracy)
    neptune.log_metric(f'test_f1_macro', test_f1_macro)

<a id="13"></a>
<h2 style='background:blue; border:0; color:white'><center>Error analysis - Confusion matrix<center><h2>

In [ ]:
# Load metadata to get classes people-friendly names
labels = df_metadata['minifigure_name'].tolist()

# Calculate confusion matrix
confusion_matrix = sk_metrics.confusion_matrix(y_test, y_pred)
# confusion_matrix = confusion_matrix / confusion_matrix.sum(axis=1)
df_confusion_matrix = pd.DataFrame(confusion_matrix, index=labels, columns=labels)

# Show confusion matrix
fig = plt.figure(figsize=(12, 12))
sn.heatmap(df_confusion_matrix, annot=True, cbar=False, cmap='Oranges', linewidths=1, linecolor='black')
plt.xlabel('Predicted labels', fontsize=15)
plt.xticks(fontsize=12)
plt.ylabel('True labels', fontsize=15)
plt.yticks(fontsize=12)

if neptune_settings["active"] and neptune_settings["log_images"]:
    neptune.log_image('error-analysis', fig)

<a id="14"></a>
<h2 style='background:blue; border:0; color:white'><center>Error analysis - Misclassified samples<center><h2>

In [ ]:
error_images = []
error_label = []
error_pred = []
error_prob = []
for batch in test_loader:
    _X_test, _y_test = batch['X'], batch['y']
    pred = torch.softmax(model(_X_test.to(device)), axis=-1).detach().cpu().numpy()
    pred_class = pred.argmax(axis=-1)
    if pred_class != _y_test.cpu().numpy():
        error_images.extend(_X_test)
        error_label.extend(_y_test)
        error_pred.extend(pred_class)
        error_prob.extend(pred.max(axis=-1))

In [ ]:
fig = plt.figure(figsize=(16, 16))
for ind, image in enumerate(error_images):
    plt.subplot(math.ceil(len(error_images) / int(len(error_images) ** 0.5)), int(len(error_images) ** 0.5), ind + 1)
    plt.imshow(denormalize_image(image.permute(1, 2, 0).numpy()))
    plt.title(f"Predict: {labels[error_pred[ind]]} ({error_prob[ind]:.2f}) Real: {labels[error_label[ind]]}")
    plt.axis("off")

if neptune_settings["active"] and neptune_settings["log_images"]:
    neptune.log_image("error-analysis", fig)

In [ ]:
if neptune_settings["active"]:
    neptune.stop()